In [1]:
# Povm

In [1]:
import pickle
import time
from typing import List, Union
import datetime as dt
from pathlib import Path
import copy
from collections import namedtuple
import json

import numpy as np
import pandas as pd

from quara.data_analysis import data_analysis
from quara.loss_function.weighted_probability_based_squared_error import (
    WeightedProbabilityBasedSquaredError,
    WeightedProbabilityBasedSquaredErrorOption,
)
from quara.loss_function.weighted_relative_entropy import (
    WeightedRelativeEntropy,
    WeightedRelativeEntropyOption,
)
from quara.minimization_algorithm.projected_gradient_descent_backtracking import (
    ProjectedGradientDescentBacktracking,
    ProjectedGradientDescentBacktrackingOption,
)
from quara.objects.composite_system import CompositeSystem
from quara.objects.elemental_system import ElementalSystem
from quara.objects.matrix_basis import get_normalized_pauli_basis
from quara.objects.qoperation import QOperation
from quara.objects.povm import Povm
from quara.objects.gate import Gate
from quara.objects.state import State
import quara.objects.qoperation_typical as qt
from quara.protocol.qtomography.standard.standard_qst import StandardQst
from quara.protocol.qtomography.standard.linear_estimator import LinearEstimator
from quara.protocol.qtomography.standard.loss_minimization_estimator import (
    LossMinimizationEstimator,
)
from quara.protocol.qtomography.standard.projected_linear_estimator import (
    ProjectedLinearEstimator,
)

from quara.simulation import standard_qtomography_simulation_report as report
from quara.simulation.standard_qtomography_simulation import (
    StandardQTomographySimulationSetting,
    execute_simulation,
)
from quara.simulation.generation_setting import QOperationGenerationSetting

from quara.simulation.standard_qtomography_simulation_check import (
    StandardQTomographySimulationCheck,
)

from quara.simulation.depolarized_qoperation_generation_setting import (
    DepolarizedQOperationGenerationSetting,
)
from quara.simulation.random_effective_lindbladian_generation_setting import (
    RandomEffectiveLindbladianGenerationSetting,
)

from quara.objects.composite_system_typical import generate_composite_system

In [2]:
%reload_ext autoreload
%autoreload 2

# Object

In [3]:
import dataclasses

@dataclasses.dataclass
class NoiseSetting:
    qoperation_base: Union[QOperation, str]
    method: str
    para: bool

@dataclasses.dataclass
class TestSetting:
    true_object: NoiseSetting
    tester_objects: List[NoiseSetting]
    seed: int
    n_rep: int
    num_data: List[int]
    n_sample: int
    schedules: Union[str, List[List[int]]]
    case_names: List[str]
    estimators: list
    algo_list: List[tuple]
    loss_list: List[tuple]
    parametrizations: List[bool]
    c_sys: CompositeSystem

@dataclasses.dataclass
class Result:
    result_index: int
    simulation_setting: StandardQTomographySimulationSetting
    estimation_results: List["EstimationResult"]
    check_result: dict

@dataclasses.dataclass
class GenerationSettings:
    true_setting: QOperationGenerationSetting
    tester_settings: List[QOperationGenerationSetting]

# Settings

In [4]:
c_sys = generate_composite_system("qubit", 1)

In [5]:
# シミュレーションに必要な設定を作る
case_names = [
    "Linear (True)",
    "Linear (False)",
    "ProjectedLinear (True)",
    "ProjectedLinear (False)",
    "Maximum-Likelihood (True)",
    "Maximum-Likelihood (False)",
    "Least Squares (True)",
    "Least Squares (False)",
]

parametrizations = [True, False, True, False, True, False, True, False]

estimators = [
    LinearEstimator(),
    LinearEstimator(),
    ProjectedLinearEstimator(),
    ProjectedLinearEstimator(),
    LossMinimizationEstimator(),
    LossMinimizationEstimator(),
    LossMinimizationEstimator(),
    LossMinimizationEstimator(),
]

loss_list = [
    (None, None),
    (None, None),
    (None, None),
    (None, None),
    (WeightedRelativeEntropy(4), WeightedRelativeEntropyOption("identity")),
    (WeightedRelativeEntropy(8), WeightedRelativeEntropyOption("identity")),
    (
        WeightedProbabilityBasedSquaredError(4),
        WeightedProbabilityBasedSquaredErrorOption("identity"),
    ),
    (
        WeightedProbabilityBasedSquaredError(8),
        WeightedProbabilityBasedSquaredErrorOption("identity"),
    ),
]


def generate_pgdb_algo_option():
    return ProjectedGradientDescentBacktrackingOption(
        mode_stopping_criterion_gradient_descent="sum_absolute_difference_variable",
        num_history_stopping_criterion_gradient_descent=1,
    )


algo_list = [
    (None, None),
    (None, None),
    (None, None),
    (None, None),
    (ProjectedGradientDescentBacktracking(), generate_pgdb_algo_option()),
    (ProjectedGradientDescentBacktracking(), generate_pgdb_algo_option()),
    (ProjectedGradientDescentBacktracking(), generate_pgdb_algo_option()),
    (ProjectedGradientDescentBacktracking(), generate_pgdb_algo_option()),
]

In [6]:
# Name of True Object
true_object_noise_setting = NoiseSetting(
    qoperation_base=("povm", "z"),
    method="random_effective_lindbladian",
    para={
        "lindbladian_base": "identity",
        "strength_h_part": 0.1,
        "strength_k_part": 0.1,
    },
)
"""
true_object_noise_setting = NoiseSetting(
   qoperation_base=("povm", "z"), method="depolarized", para={"error_rate": 0.1,},
)
"""

# Name of Tester Object
tester_names = [("state", name) for name in ["x0", "y0", "z0", "z1"]]

tester_object_noise_settings = [
    NoiseSetting(
        qoperation_base=name,
        method="random_effective_lindbladian",
        para={
            "lindbladian_base": "identity",
            "strength_h_part": 0.1,
            "strength_k_part": 0.1,
        },
    )
    for name in tester_names
]

"""
tester_object_noise_settings = [
   NoiseSetting(qoperation_base=name, method="depolarized", para={"error_rate": 0.1,},)
   for name in tester_names
]
"""

# Test Setting
test_setting_0 = TestSetting(
    true_object=true_object_noise_setting,
    tester_objects=tester_object_noise_settings,
    seed=777,
    n_sample=2,
    n_rep=10,
    num_data=[10, 100],
    schedules="all",  # 全ての組み合わせ
    case_names=case_names,
    estimators=estimators,
    algo_list=algo_list,
    loss_list=loss_list,
    parametrizations=parametrizations,
    c_sys=c_sys,
)

# TODO: 複数のtest_settingをtest_settingsに格納する
# 時間短縮のため、今は1つだけ格納する
test_settings = [test_setting_0]

# Functions

In [7]:
# 1つのtest_settingから、QOperationGenerationSettingのリストを作る
def to_generation_setting(noise_setting: NoiseSetting, c_sys: CompositeSystem) -> "GenerationSetting":
    # TODO: 他に良い方法がないか検討
    name2class_map = {
        "depolarized": DepolarizedQOperationGenerationSetting,
        "random_effective_lindbladian": RandomEffectiveLindbladianGenerationSetting,
    }

    if noise_setting.method in name2class_map:
        target_class = name2class_map[noise_setting.method]
    else:
        message = f"noise_setting.method='{noise_setting.method}' is not implemented."
        raise NotImplementedError(message)
    return target_class(
        qoperation_base=noise_setting.qoperation_base,
        c_sys=c_sys,
        **noise_setting.para,
    )


def to_generation_settings(test_setting: TestSetting) -> GenerationSettings:
    true_setting = to_generation_setting(test_setting.true_object, test_setting.c_sys)
    tester_settings = [
        to_generation_setting(setting, test_setting.c_sys) for setting in test_setting.tester_objects
    ]
    generation_settings = GenerationSettings(
        true_setting=true_setting, tester_settings=tester_settings
    )
    return generation_settings


def to_simulation_setting(
    test_setting: TestSetting,
    true_object: "QOperation",
    tester_objects: List["QOperation"],
    case_index: int,
) -> StandardQTomographySimulationSetting:
    return StandardQTomographySimulationSetting(
        name=test_setting.case_names[case_index],
        estimator=test_setting.estimators[case_index],
        loss=test_setting.loss_list[case_index][0],
        loss_option=test_setting.loss_list[case_index][1],
        algo=test_setting.algo_list[case_index][0],
        algo_option=test_setting.algo_list[case_index][1],
        true_object=true_object,
        tester_objects=tester_objects,
        n_rep=test_setting.n_rep,
        seed=test_setting.seed,
        num_data=test_setting.num_data,
        schedules=test_setting.schedules,
    )

In [8]:
from quara.protocol.qtomography.standard.standard_qst import StandardQst
from quara.protocol.qtomography.standard.standard_qpt import StandardQpt
from quara.protocol.qtomography.standard.standard_povmt import StandardPovmt


def generate_qtomography(
    sim_setting: StandardQTomographySimulationSetting,
    para: bool,
    eps_proj_physical: float,
) -> "StandardQTomography":
    # TrueObjectに応じて、適切なQTomographyを生成する
    true_object = sim_setting.true_object
    tester_objects = sim_setting.tester_objects
    seed = sim_setting.seed

    if type(true_object) == State:
        return StandardQst(
            tester_objects,
            on_para_eq_constraint=para,
            seed=seed,
            eps_proj_physical=eps_proj_physical,
        )
    if type(true_object) == Povm:
        return StandardPovmt(
            tester_objects,
            on_para_eq_constraint=para,
            seed=seed,
            eps_proj_physical=eps_proj_physical,
            num_outcomes=len(true_object.vecs)
        )
    if type(true_object) == Gate:
        states = [t for t in tester_objects if type(t) == State]
        povms = [t for t in tester_objects if type(t) == Povm]

        return StandardQpt(
            states=states,
            povms=povms,
            on_para_eq_constraint=para,
            seed=seed,
            eps_proj_physical=eps_proj_physical,
        )
    message = f"type of sim_setting.true_object must be State, Povm, or Gate, not {type(true_object)}"
    print(f"{sim_setting.true_object}")
    raise TypeError(message)

In [9]:
def result2dict(result: Result) -> dict:
    result_dict = dict(
        test_setting_index=result.result_index["test_setting_index"],
        sample_index=result.result_index["sample_index"],
        case_index=result.result_index["case_index"],
        name=result.simulation_setting.name,
        total_result=result.check_result["total_result"],
    )

    def _make_warning_text(r):
        possibly_ok = r["detail"]["possibly_ok"]
        to_be_checked = r["detail"]["to_be_checked"]
        warning_text = ""
        if not possibly_ok:
            warning_text = (
                f"Consistency: possibly_ok={possibly_ok}, to_be_checked={to_be_checked}"
            )
        return warning_text

    check_result = {}
    warning_text = ""

    for r in result.check_result["results"]:
        check_result[r["name"]] = r["result"]
        if r["name"] == "Consistency":
            warning_text += _make_warning_text(r)

    result_dict.update(check_result)
    result_dict["warning"] = warning_text

    return result_dict

In [10]:
def write_setting(test_setting: TestSetting, root_dir: str, test_setting_index: int) -> None:
    dir_path = Path(f"{root_dir}/{test_setting_index}/")
    dir_path.mkdir(parents=True, exist_ok=True)
    path = dir_path / f"test_setting.pickle"
    with open(path, "wb") as f:
        pickle.dump(test_setting, f)

    print(f"Completed to write test_setting. {path}")

def write_results(results: List[Result], dir_path: str) -> None:
    dir_path = Path(dir_path)
    dir_path.mkdir(parents=True, exist_ok=True)
    path = dir_path / "check_result.csv"

    result_dict_list = [result2dict(r) for r in results]
    sample_result_df = pd.DataFrame(result_dict_list)
    sample_result_df.to_csv(path, index=None)

    print(f"Completed to write csv. {path}")


def write_result_case_unit(result: namedtuple, root_dir: str) -> None:
    test_setting_index = result.result_index["test_setting_index"]
    sample_index = result.result_index["sample_index"]
    case_index = result.result_index["case_index"]

    # Save all
    dir_path = Path(f"{root_dir}/{test_setting_index}/{sample_index}")
    dir_path.mkdir(parents=True, exist_ok=True)
    path = dir_path / f"case_{case_index}_result.pickle"
    with open(path, "wb") as f:
        pickle.dump(result, f)

    check_result = result.check_result
    path = dir_path / f"case_{case_index}_check_result.json"
    with open(path, "w") as f:
        json.dump(check_result, f, ensure_ascii=False, indent=4, separators=(",", ": "))


def write_result_sample_unit(results: List[namedtuple], root_dir: str) -> None:
    test_setting_index = results[0].result_index["test_setting_index"]
    sample_index = results[0].result_index["sample_index"]
    dir_path = Path(root_dir) / str(test_setting_index) / str(sample_index)

    write_results(results, dir_path)


def write_result_test_setting_unit(results: List[namedtuple], root_dir: str) -> None:
    test_setting_index = results[0].result_index["test_setting_index"]
    dir_path = Path(root_dir) / str(test_setting_index)

    write_results(results, dir_path)


def write_pdf_report(results: List[Result], root_dir: str) -> None:
    test_setting_index = results[0].result_index["test_setting_index"]
    sample_index = results[0].result_index["sample_index"]

    dir_path = Path(root_dir) / str(test_setting_index) / str(sample_index)
    dir_path.mkdir(parents=True, exist_ok=True)
    path = dir_path / f"{test_setting_index}_{sample_index}_quara_report.pdf"

    estimation_results_list = [r.estimation_results for r in results]
    sim_settings = [r.simulation_setting for r in results]

    report.export_report(path, estimation_results_list, sim_settings)

In [11]:
def execute_simulation_case_unit(
    test_setting,
    true_object,
    tester_objects,
    case_index: int,
    sample_index: int,
    test_setting_index: int,
    root_dir: str,
) -> Result:
    # Generate QTomographySimulationSetting
    sim_setting = to_simulation_setting(
        test_setting, true_object, tester_objects, case_index
    )
    print(f"Case {case_index}: {sim_setting.name}")

    def _copy_sim_setting(source):
        return StandardQTomographySimulationSetting(
            name=source.name,
            true_object=source.true_object,
            tester_objects=source.tester_objects,
            estimator=source.estimator,
            loss=copy.deepcopy(source.loss),
            loss_option=source.loss_option,
            algo=copy.deepcopy(source.algo),
            algo_option=source.algo_option,
            seed=source.seed,
            n_rep=source.n_rep,
            num_data=source.num_data,
            schedules=source.schedules,
        )

    org_sim_setting = _copy_sim_setting(sim_setting)

    # QTomographyの作成
    qtomography = generate_qtomography(
        sim_setting,
        para=test_setting.parametrizations[case_index],
        eps_proj_physical=1e-13,
    )

    # 実行
    estimation_results = execute_simulation(
        qtomography=qtomography, simulation_setting=sim_setting
    )

    # 自動テスト
    sim_check = StandardQTomographySimulationCheck(sim_setting, estimation_results)
    check_result = sim_check.execute_all(show_detail=False, with_detail=True)

    # 結果の表示
    if not check_result["total_result"]:
        start_red = "\033[31m"
        end_color = "\033[0m"
        print(f"Total Result: {start_red}NG{end_color}")

    result_index = dict(
        test_setting_index=test_setting_index,
        sample_index=sample_index,
        case_index=case_index,
    )

    # 結果の格納
    result = Result(
        result_index=result_index,
        simulation_setting=org_sim_setting,
        estimation_results=estimation_results,
        check_result=check_result,
    )
    # Save
    write_result_case_unit(result, root_dir=root_dir)
    return result

In [19]:
def execute_simulation_sample_unit(
    test_setting,
    generation_settings,
    test_setting_index,
    sample_index,
    root_dir,
    pdf_mode: str = "only_ng",
) -> List[Result]:
    # sampleの生成
    true_object = generation_settings.true_setting.generate()
    true_object = true_object[0] if type(true_object) == tuple else true_object
    tester_objects = [
        tester_setting.generate() 
        for tester_setting in generation_settings.tester_settings
    ]
    tester_objects = [tester[0] if type(tester) == tuple else tester for tester in tester_objects]
    
    # TODO: remove codes for debugging
    # ランダムに生成されたコードがフィジカルかどうかを確認する
    # print(f"true_object = {true_object.is_()}")
    # print(true_object)
    # print(f"tester_object")
    # for t in tester_objects:
    #    print("=================")
    #    print(t.is_physical())
    #    print(t)

    results = []
    case_n = len(test_setting.case_names)

    for case_index in range(case_n):
        result = execute_simulation_case_unit(
            test_setting,
            true_object=true_object,
            tester_objects=tester_objects,
            case_index=case_index,
            sample_index=sample_index,
            test_setting_index=test_setting_index,
            root_dir=root_dir,
        )
        results.append(result)

    # Save
    write_result_sample_unit(results, root_dir=root_dir)

    # Save PDF
    if pdf_mode == "all":
        write_pdf_report(results, root_dir)
    elif pdf_mode == "only_ng":
        total_results = [r.check_result["total_result"] for r in results]
        print(f"total_result={np.all(total_results)}")
        if not np.all(total_results):
            write_pdf_report(results, root_dir)
    elif pdf_mode == "none":
        pass
    else:
        message = "`pdf_mode` must be 'all', 'only_ng', or 'none'."
        raise ValueError(message)

    return results


def execute_simulation_test_setting_unit(
    test_setting, test_setting_index, dir_path, pdf_mode: str = "only_ng"
) -> List[Result]:
    generation_settings = to_generation_settings(test_setting)
    n_sample = test_setting.n_sample
    results = []

    for sample_index in range(n_sample):
        sample_results = execute_simulation_sample_unit(
            test_setting,
            generation_settings,
            test_setting_index,
            sample_index,
            root_dir,
            pdf_mode=pdf_mode,
        )
        results += sample_results

    # 保存（test_index単位）
    write_result_test_setting_unit(results, dir_path)
    return results

In [20]:
def print_summary(results: List[Result], elapsed_time: float) -> None:
    result_dict_list = [result2dict(r) for r in results]
    result_df = pd.DataFrame(result_dict_list)
    case_n = result_df.shape[0]
    ok_n = result_df[result_df["total_result"]].shape[0]
    ng_n = result_df[~result_df["total_result"]].shape[0]
    warning_n = result_df[
        result_df["total_result"] & result_df["warning"].isnull()
    ].shape[0]

    def _to_h_m_s(sec) -> tuple:
        m, s = divmod(int(sec), 60)
        h, m = divmod(m, 60)
        return h, m, s

    h, m, s = _to_h_m_s(elapsed_time)
    time_text = "{:.1f}s ".format(elapsed_time)
    time_text += f"({h}:{str(m).zfill(2)}:{str(s).zfill(2)})"

    start_red = "\033[31m"
    start_green = "\033[32m"
    start_yellow = "\033[33m"
    end_color = "\033[0m"

    summary_text = f"{start_yellow}=============={end_color} "
    summary_text += f"{start_green}OK: {ok_n} cases{end_color} ({start_yellow}{warning_n} warnings{end_color}), "
    summary_text += f"{start_red}NG: {ng_n} cases{end_color} "
    summary_text += f"{start_yellow} in {time_text}s=============={end_color}"

    print(summary_text)

# Execute

In [21]:
def execute_simulation_test_settings(test_settings: List[TestSetting], root_dir: str) -> List[Result]:
    all_results = []
    start = time.time()

    for test_setting_index, test_setting in enumerate(test_settings):
        write_setting(test_setting, root_dir, test_setting_index)
        test_results = execute_simulation_test_setting_unit(
            test_setting, test_setting_index, root_dir, pdf_mode="only_ng"
        )
        all_results += test_results

    # Save
    write_results(all_results, root_dir)

    elapsed_time = time.time() - start
    print_summary(all_results, elapsed_time)
    
    return all_results

In [25]:
root_dir = "result_random_povmt"
all_results = execute_simulation_test_settings(test_settings, root_dir)

Completed to write test_setting. result_random_povmt/0/test_setting.pickle


/Users/tomoko/project/rcast/workspace/quara/quara/utils/matrix_util.py:195: ComplexWarning:

Casting complex values to real discards the imaginary part

/Users/tomoko/project/rcast/workspace/quara/quara/objects/effective_lindbladian.py:723: ComplexWarning:

Casting complex values to real discards the imaginary part

 60%|██████    | 6/10 [00:00<00:00, 59.75it/s]

Case 0: Linear (True)


100%|██████████| 10/10 [00:00<00:00, 12347.08it/s]

========== Summary ============
Name: Linear (True)
MSE of Empirical Distributions: OK
Consistency: OK (to_be_checked=False)
MSE of estimators: OK
Physicality Violation: OK
Case 1: Linear (False)


...

​Generating a graph for MSE ...


100%|██████████| 10/10 [00:00<00:00, 10330.80it/s]

​​Generating physicality violation test blocks ...



  0%|          | 0/2 [00:00<?, ?it/s]
2it [00:00, 64.56it/s]

2it [00:00, 86.50it/s]
  0%|          | 0/2 [00:00<?, ?it/s]
2it [00:00, 57.01it/s]

2it [00:00, 90.43it/s]
  0%|          | 0/2 [00:00<?, ?it/s]
2it [00:00, 62.00it/s]

2it [00:00, 81.87it/s]
  0%|          | 0/2 [00:00<?, ?it/s]
2it [00:00, 60.64it/s]

2it [00:00, 79.80it/s]
  0%|          | 0/2 [00:00<?, ?it/s]
2it [00:00, 74.07it/s]

2it [00:00, 81.64it/s]
  0%|          | 0/2 [00:00<?, ?it/s]
2it [00:00, 70.09it/s]

2it [00:00, 83.08it/s]
  0%|          | 0/2 [00:00<?, ?it/s]
2it [00:00, 80.37it/s]

2it [00:00, 88.10it/s]
  0%|          | 0/2 [00:00<?, ?it/s]
2it [00:00, 57.90it/s]

2it [00:00, 82.82it/s]
  0%|          | 0/2 [00:00<?, ?it/s]
2it [00:00, 54.36it/s]

2it [00:00, 67.57it/s]
  0%|          | 0/2 [00:00<?, ?it/s]
2it [00:00, 68.34it/s]

2it [00:00, 38.69it/s]
  0%|          | 0/2 [00:00<?, ?it/s]
2it [00:00, 64.75it/s]

2it [00:00, 82.47it/s]
  0%|          | 0/2 [00:00<?, ?it/s]
2it [00:00, 82.59it/s]

2i

Converting to PDF report ...
​Deleting temporary files ...
Completed to export pdf. (result_random_povmt/0/1/0_1_quara_report.pdf)
Completed to write csv. result_random_povmt/0/check_result.csv
Completed to write csv. result_random_povmt/check_result.csv
============== OK: 8 cases (0 warnings), NG: 8 cases  in 614.8s (0:10:14)s==============
